In [2]:
import pandas as pd 
import os 


In [3]:
df = pd.read_csv("/home/mohammed/Downloads/nb.csv") 

In [4]:
df.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [5]:
df.count()

Trip ID             536020
Trip Origin         536020
Trip Destination    536020
Trip Start Time     534369
Trip End Time       536019
dtype: int64

In [6]:
df.isnull().sum()

Trip ID                0
Trip Origin            0
Trip Destination       0
Trip Start Time     1651
Trip End Time          1
dtype: int64

In [7]:
db = df.dropna()

In [8]:
db.count()

Trip ID             534368
Trip Origin         534368
Trip Destination    534368
Trip Start Time     534368
Trip End Time       534368
dtype: int64

In [ ]:
db['Trip End Time'] = pd.to_datetime(db['Trip End Time'])
db['Trip End Time'] = pd.to_datetime(db['Trip End Time'])

In [11]:
df['trip_time']= (df['Trip End Time']- df['Trip Start Time']).dt.total_seconds()
df.head()

TypeError: unsupported operand type(s) for -: 'str' and 'str'